In [3]:
!pip install tensorflow

   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 2.6/376.0 MB 15.1 MB/s eta 0:00:25
    --------------------------------------- 5.2/376.0 MB 13.9 MB/s eta 0:00:27
    --------------------------------------- 8.4/376.0 MB 14.1 MB/s eta 0:00:27
   - -------------------------------------- 12.3/376.0 MB 15.4 MB/s eta 0:00:24
   - -------------------------------------- 16.0/376.0 MB 16.0 MB/s eta 0:00:23
   -- ------------------------------------- 19.9/376.0 MB 16.6 MB/s eta 0:00:22
   -- ------------------------------------- 23.6/376.0 MB 16.8 MB/s eta 0:00:21
   -- ------------------------------------- 27.3/376.0 MB 16.9 MB/s eta 0:00:21
   --- ------------------------------------ 31.2/376.0 MB 17.1 MB/s eta 0:00:21
   --- ------------------------------------ 35.4/376.0 MB 17.3 MB/s eta 0:00:20
   ---- ----------------------------------- 39.1/376.0 MB 17.5 MB/s eta 0:00:20
   ---- ----------------------------------- 43.0/376

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.utils.class_weight import compute_class_weight

In [17]:
train_dir = "C:/Users/Deepa/OneDrive/Documents/oral_detection/Oral_Cancer_dataset"
img_size = (128,128)
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)
val_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

Found 601 images belonging to 2 classes.
Found 150 images belonging to 2 classes.


In [19]:
print("Class indices:",train_data.class_indices)

Class indices: {'CANCER': 0, 'NON CANCER': 1}


In [29]:
class_weights = compute_class_weight(
    class_weight = "balanced",
    classes = np.unique(train_data.classes),
    y = train_data.classes
)
class_weights = dict(enumerate(class_weights))

In [33]:
model = Sequential([
    Conv2D(32,(3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

C:\Users\Deepa\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,449 (28.20 MB)

 Trainable params: 7,392,449 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights
)

C:\Users\Deepa\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
18/19 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4502 - loss: 2.5764

C:\Users\Deepa\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.4503 - loss: 2.4666 - val_accuracy: 0.6933 - val_loss: 0.6696
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6926 - loss: 0.6659 - val_accuracy: 0.6333 - val_loss: 0.6531
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 1000ms/step - accuracy: 0.7116 - loss: 0.5701 - val_accuracy: 0.7467 - val_loss: 0.5439
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7886 - loss: 0.4871 - val_accuracy: 0.7133 - val_loss: 0.5941
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 997ms/step - accuracy: 0.8253 - loss: 0.3966 - val_accuracy: 0.7600 - val_loss: 0.5340
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8654 - loss: 0.3551 - val_accuracy: 0.7267 - val_loss: 0.5388
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8981 - loss: 0.2815 - val_accuracy: 0.7667 - val_loss: 0.5342
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8891 - loss: 0.2832 - val_accuracy: 0.7600 - val_loss: 0.5

In [41]:
model.save('oral_cancer_cnn.h5')